In [1]:
import urllib.request
import gzip
import json

DATA_URL = "http://ciir.cs.umass.edu/downloads/poetry/poetry50k.dedup.jsonl.gz"
data_50k = []


def read_data(target):
    handle = urllib.request.urlopen(DATA_URL)
    with gzip.GzipFile(fileobj=handle) as fp:
        for line in fp:
            content = json.loads(line)
            if content["score"]>0.65:
                target.append(content["text"]) 
                
read_data(data_50k)

In [2]:
data_50k = data_50k[:50000]

In [3]:
data_50k[0]

'^Dedication\nHere\ts\ta\trhyme\tfor\tBarbara,\nLaughing\twhite\tand\tpink,\nHere\ts\ta\trhyme\tfor\tsmiling\tTed,\nAnd\tone\tfor\tWink.\nNow\tDick\ts\tnot\tmuch\tat\treading\trhymes,\nHe\td\trather\tsit\tand\tfish.\nWell,\there\ts\ta\tcouple\tof\tverses,\tDick,\nRead\tthem\tif\tyou\twish!\n479468\n'

In [4]:
data_1000=data_50k[:100]

In [11]:
import re
line_break = re.compile(r"(\n|<br />)")


def build_dictionary(database, weights):
    
    dictionary = {}
    sentence_length_dictionary={}
    
    punctuations = '''!()-+='[]{};:"\,<>./?@#$%^&*_~1234567890«»|'''
    translator = str.maketrans('', '', punctuations)
    prev_words=["","","",""]

    for poem in database:
        lines = re.split(line_break, poem)

        # first lines are usually titles
        for line in lines[1:]:    
            line = line.replace("\n","")
            line = line.translate(translator)
            newline = True          
            index = 0
            words = line.split('\t')
            for word in words:
                if word:
                    if not word == "I":
                        word = word.lower()

                    if newline:

                        if len(words) in sentence_length_dictionary:
                            sentence_length_dictionary[len(words)] +=1
                        else:
                            sentence_length_dictionary[len(words)] = 1

                        for _ in prev_words:
                            _ = ""
                            
                        newline = False

                    if word in dictionary:
                        dictionary[word]["frequency"] += 1
                    else:
                        dictionary[word] = {
                                            "frequency":1,
                                            "dict":{}
                                            }

                    if index >= 2 and not prev_words[1] == "":
                        if prev_words[1] in dictionary:

                            prev_dict2 = dictionary[prev_words[1]]["dict"]
                            if str((prev_words[0],word)) in prev_dict2:
                                prev_dict2[str((prev_words[0],word))] += weights[0]
                            else:
                                prev_dict2[str((prev_words[0],word))] = weights[0]

                        else:
                            print("preprevious word not in dictionary error")

                    if index >=3 and not prev_words[2] == "":

                        if prev_words[2] in dictionary :

                            prev_dict3 = dictionary[prev_words[2]]["dict"]
                            if str((prev_words[1], prev_words[0], word)) in prev_dict3:
                                prev_dict3[str((prev_words[1], prev_words[0], word))] += weights[1]
                            else: 
                                prev_dict3[str((prev_words[1], prev_words[0], word))] = weights[1]
                                
                    if index >=4 and not prev_words[3] == "":

                        if prev_words[3] in dictionary:

                            prev_dict4 = dictionary[prev_words[3]]["dict"]
                            if str((prev_words[2], prev_words[1], prev_words[0], word)) in prev_dict4:
                                prev_dict4[str((prev_words[2], prev_words[1], prev_words[0], word))] += weights[2]
                            else: 
                                prev_dict4[str((prev_words[2], prev_words[1], prev_words[0], word))] = weights[2]

                    
                    for i in range(len(prev_words)-1):
                        prev_words[i+1] = prev_words[i]
                    prev_words[0] = word
                    
                    index+=1
                    
    return dictionary, sentence_length_dictionary

dictionary_1k,sl_1k = build_dictionary(data_1000, [1,2,6])


In [12]:
with open('dictionary1000.json',"w") as out_file:  
    json.dump(dictionary_1k, out_file, sort_keys=True, indent=4)

with open('sentence_lengthes_1k.json',"w") as out_file:  
    json.dump(sl_1k, out_file, sort_keys=True, indent=4)


In [ ]:
def new_senetnce(dictionary, sentence_len_freq):

    sentence = ""
    sentence_length = random.choices(list(sentence_len_freq.keys()), weights = sentence_len_freq.values(), k=1)[0]
    # index a the word in the current sentence
    index = 0

    word_frequencies = []
    for _ in dictionary.values():
        word_frequencies.append(_["frequency"])
        
    first_word = random.choices(list(dictionary.keys()), weights = word_frequencies, k=1)[0]
    sentence+=first_word.capitalize()
    current_words = [first_word]

    while(index < sentence_length-2):
        
        key = current_words[-1]
        while type(key) != str:
            key = key[-1]

        current_dictionary = dictionary[key]["dict"]

        if current_dictionary:
            current_words = random.choices(list(current_dictionary.keys()),weights = current_dictionary.values(), k=1)
            print(current_words)
        else: 
            break


        for _ in current_words:
            for __ in _[1:]:
                sentence = sentence + " " + __
                index += 1
        
    return sentence

In [ ]:
def generate_poem(length, dictionary, sentence_len_freq):
    
    for _ in range(length):
        sentence = new_senetnce(dictionary, sentence_len_freq)
        print(sentence)


In [ ]:
generate_poem(15, dictionary_50k, sentence_lengthes_50k)